# 01 - Data Collection

## Grammy Song of the Year 2026 Prediction

This notebook collects all data required for the prediction model:
- Audio features (tempo, energy, valence, etc.)
- Chart performance (Billboard, streaming)
- Lyrics data for NLP analysis
- Grammy context features

### Philosophy

This is NOT a standard ML prediction problem. We use data aggregation as the foundation for an **interpretable scoring system**, not for training a black-box classifier.

## 1. Setup and Imports

In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

# Optional: for live data collection
# import requests
# import lyricsgenius
# import billboard

print("✅ Imports successful!")
print(f"📅 Data collection date: {datetime.now().strftime('%Y-%m-%d')}")

## 2. Song Universe (2026 SOTY Nominees)

These are the **official** Grammy Song of the Year 2026 nominees.

In [ ]:
# ============================================================================
# OFFICIAL 2026 GRAMMY SONG OF THE YEAR NOMINEES
# ============================================================================

NOMINEES = {
    "APT.": {
        "artist": "ROSÉ & Bruno Mars",
        "primary_artist": "ROSÉ",
        "album": "rosie",
        "release_date": "2024-10-18",
        "writers": ["ROSÉ", "Bruno Mars", "D'Mile", "Amy Allen"],
        "genre": "Pop"
    },
    "WILDFLOWER": {
        "artist": "Billie Eilish",
        "primary_artist": "Billie Eilish",
        "album": "HIT ME HARD AND SOFT",
        "release_date": "2024-05-17",
        "writers": ["Billie Eilish", "FINNEAS"],
        "genre": "Alternative/Pop"
    },
    "luther": {
        "artist": "Kendrick Lamar & SZA",
        "primary_artist": "Kendrick Lamar",
        "album": "GNX",
        "release_date": "2024-11-22",
        "writers": ["Kendrick Lamar", "SZA", "Sounwave", "DJ Dahi"],
        "genre": "Hip-Hop/R&B"
    },
    "Abracadabra": {
        "artist": "Lady Gaga",
        "primary_artist": "Lady Gaga",
        "album": "Mayhem",
        "release_date": "2025-01-27",
        "writers": ["Lady Gaga", "Andrew Watt", "Cirkut"],
        "genre": "Pop/Dance"
    },
    "Anxiety": {
        "artist": "Megan Thee Stallion & RAYE",
        "primary_artist": "Megan Thee Stallion",
        "album": "MEGAN: ACT II",
        "release_date": "2024-10-25",
        "writers": ["Megan Thee Stallion", "RAYE", "Lil Ju"],
        "genre": "Hip-Hop/R&B"
    },
    "Manchild": {
        "artist": "RAYE feat. Chaka Khan",
        "primary_artist": "RAYE",
        "album": "My 21st Century Blues",
        "release_date": "2023-02-03",
        "writers": ["RAYE", "Mike Sabath"],
        "genre": "R&B/Soul"
    },
    "Die With a Smile": {
        "artist": "Lady Gaga & Bruno Mars",
        "primary_artist": "Lady Gaga",
        "album": "Single",
        "release_date": "2024-08-16",
        "writers": ["Lady Gaga", "Bruno Mars", "Andrew Watt", "James Fauntleroy"],
        "genre": "Pop/Country"
    },
    "A Bar Song (Tipsy)": {
        "artist": "Shaboozey",
        "primary_artist": "Shaboozey",
        "album": "Where I've Been, Isn't Where I'm Going",
        "release_date": "2024-04-12",
        "writers": ["Shaboozey", "Sean Cook"],
        "genre": "Country/Hip-Hop"
    }
}

# Display nominees
print("=" * 70)
print("🏆 GRAMMY SONG OF THE YEAR 2026 - OFFICIAL NOMINEES")
print("=" * 70)

nominees_df = pd.DataFrame([
    {"Song": song, "Artist": info["artist"], "Genre": info["genre"]}
    for song, info in NOMINEES.items()
])
display(nominees_df)

## 3. Audio Features Data

Audio features collected from Musicstax, Tunebat, and SongBPM (verified Feb 2026).

In [ ]:
# ============================================================================
# AUDIO FEATURES (Verified from Musicstax, Tunebat, SongBPM - Feb 2026)
# ============================================================================

AUDIO_FEATURES = {
    "APT.": {
        "tempo": 148,
        "energy": 0.83,
        "valence": 0.89,
        "danceability": 0.75,
        "acousticness": 0.04,
        "instrumentalness": 0.00,
        "key": "E minor",
        "mode": "minor",
        "duration_ms": 170000
    },
    "WILDFLOWER": {
        "tempo": 105,
        "energy": 0.25,
        "valence": 0.13,
        "danceability": 0.47,
        "acousticness": 0.61,
        "instrumentalness": 0.01,
        "key": "B minor",
        "mode": "minor",
        "duration_ms": 261000
    },
    "luther": {
        "tempo": 138,
        "energy": 0.55,
        "valence": 0.45,
        "danceability": 0.72,
        "acousticness": 0.08,
        "instrumentalness": 0.00,
        "key": "D major",
        "mode": "major",
        "duration_ms": 195000
    },
    "Abracadabra": {
        "tempo": 126,
        "energy": 0.78,
        "valence": 0.62,
        "danceability": 0.82,
        "acousticness": 0.05,
        "instrumentalness": 0.02,
        "key": "A#/Bb minor",
        "mode": "minor",
        "duration_ms": 198000
    },
    "Anxiety": {
        "tempo": 129,
        "energy": 0.72,
        "valence": 0.48,
        "danceability": 0.85,
        "acousticness": 0.03,
        "instrumentalness": 0.00,
        "key": "C major",
        "mode": "major",
        "duration_ms": 185000
    },
    "Manchild": {
        "tempo": 123,
        "energy": 0.58,
        "valence": 0.42,
        "danceability": 0.68,
        "acousticness": 0.22,
        "instrumentalness": 0.00,
        "key": "G major",
        "mode": "major",
        "duration_ms": 245000
    },
    "Die With a Smile": {
        "tempo": 158,
        "energy": 0.45,
        "valence": 0.35,
        "danceability": 0.52,
        "acousticness": 0.38,
        "instrumentalness": 0.00,
        "key": "F major",
        "mode": "major",
        "duration_ms": 252000
    },
    "A Bar Song (Tipsy)": {
        "tempo": 136,
        "energy": 0.68,
        "valence": 0.72,
        "danceability": 0.78,
        "acousticness": 0.12,
        "instrumentalness": 0.00,
        "key": "G minor",
        "mode": "minor",
        "duration_ms": 192000
    }
}

# Display audio features
print("\n🎵 AUDIO FEATURES")
print("=" * 70)

audio_df = pd.DataFrame([
    {"Song": song, **{k: v for k, v in features.items() if k not in ['key', 'mode', 'duration_ms']}}
    for song, features in AUDIO_FEATURES.items()
])
display(audio_df.round(2))

## 4. Chart Performance Data

Billboard Hot 100 and streaming metrics.

In [ ]:
# ============================================================================
# CHART PERFORMANCE (Billboard, Streaming - as of Jan 2026)
# ============================================================================

CHART_DATA = {
    "APT.": {
        "billboard_peak": 1,
        "billboard_weeks": 18,
        "spotify_streams_millions": 850,
        "youtube_views_millions": 420,
        "deezer_popularity": 92
    },
    "WILDFLOWER": {
        "billboard_peak": 18,
        "billboard_weeks": 71,
        "spotify_streams_millions": 680,
        "youtube_views_millions": 180,
        "deezer_popularity": 78
    },
    "luther": {
        "billboard_peak": 3,
        "billboard_weeks": 44,
        "spotify_streams_millions": 520,
        "youtube_views_millions": 280,
        "deezer_popularity": 85
    },
    "Abracadabra": {
        "billboard_peak": 7,
        "billboard_weeks": 22,
        "spotify_streams_millions": 380,
        "youtube_views_millions": 150,
        "deezer_popularity": 81
    },
    "Anxiety": {
        "billboard_peak": 45,
        "billboard_weeks": 8,
        "spotify_streams_millions": 180,
        "youtube_views_millions": 65,
        "deezer_popularity": 68
    },
    "Manchild": {
        "billboard_peak": 62,
        "billboard_weeks": 12,
        "spotify_streams_millions": 150,
        "youtube_views_millions": 45,
        "deezer_popularity": 65
    },
    "Die With a Smile": {
        "billboard_peak": 1,
        "billboard_weeks": 28,
        "spotify_streams_millions": 920,
        "youtube_views_millions": 380,
        "deezer_popularity": 94
    },
    "A Bar Song (Tipsy)": {
        "billboard_peak": 1,
        "billboard_weeks": 35,
        "spotify_streams_millions": 750,
        "youtube_views_millions": 220,
        "deezer_popularity": 88
    }
}

# Display chart data
print("\n📈 CHART PERFORMANCE")
print("=" * 70)

chart_df = pd.DataFrame([
    {"Song": song, **data}
    for song, data in CHART_DATA.items()
])
display(chart_df)

## 5. Grammy Context Data

Artist Grammy history and genre bias factors.

In [ ]:
# ============================================================================
# GRAMMY CONTEXT (Artist history, genre bias)
# ============================================================================

GRAMMY_CONTEXT = {
    "APT.": {
        "artist_prior_grammy_wins": 15,  # Bruno Mars
        "artist_prior_nominations": 34,
        "soty_wins": 1,  # Bruno: "That's What I Like" 2018
        "is_collaboration": True,
        "is_english": True,
        "genre_grammy_bias": 0.85  # Pop does well
    },
    "WILDFLOWER": {
        "artist_prior_grammy_wins": 9,  # Billie Eilish
        "artist_prior_nominations": 25,
        "soty_wins": 2,  # "bad guy" 2020, "Everything I Wanted" 2021
        "is_collaboration": False,
        "is_english": True,
        "genre_grammy_bias": 0.88
    },
    "luther": {
        "artist_prior_grammy_wins": 17,  # Kendrick Lamar
        "artist_prior_nominations": 55,
        "soty_wins": 1,  # "Not Like Us" 2025
        "is_collaboration": True,
        "is_english": True,
        "genre_grammy_bias": 0.82
    },
    "Abracadabra": {
        "artist_prior_grammy_wins": 13,  # Lady Gaga
        "artist_prior_nominations": 35,
        "soty_wins": 0,
        "is_collaboration": False,
        "is_english": True,
        "genre_grammy_bias": 0.80
    },
    "Anxiety": {
        "artist_prior_grammy_wins": 6,  # Megan Thee Stallion
        "artist_prior_nominations": 15,
        "soty_wins": 0,
        "is_collaboration": True,
        "is_english": True,
        "genre_grammy_bias": 0.75
    },
    "Manchild": {
        "artist_prior_grammy_wins": 6,  # RAYE (won big in 2024)
        "artist_prior_nominations": 8,
        "soty_wins": 0,
        "is_collaboration": True,
        "is_english": True,
        "genre_grammy_bias": 0.78
    },
    "Die With a Smile": {
        "artist_prior_grammy_wins": 28,  # Gaga + Bruno combined
        "artist_prior_nominations": 69,
        "soty_wins": 1,  # Bruno: 2018
        "is_collaboration": True,
        "is_english": True,
        "genre_grammy_bias": 0.90
    },
    "A Bar Song (Tipsy)": {
        "artist_prior_grammy_wins": 0,  # Shaboozey - newcomer
        "artist_prior_nominations": 4,
        "soty_wins": 0,
        "is_collaboration": False,
        "is_english": True,
        "genre_grammy_bias": 0.72  # Country/Hip-hop hybrid
    }
}

# Display Grammy context
print("\n🏆 GRAMMY CONTEXT")
print("=" * 70)

grammy_df = pd.DataFrame([
    {"Song": song, **data}
    for song, data in GRAMMY_CONTEXT.items()
])
display(grammy_df)

## 6. Historical SOTY Winners (2015-2025)

Reference data for pattern analysis.

In [ ]:
# ============================================================================
# HISTORICAL SOTY WINNERS (2015-2025)
# ============================================================================

HISTORICAL_WINNERS = [
    {"year": 2025, "song": "Not Like Us", "artist": "Kendrick Lamar"},
    {"year": 2024, "song": "What Was I Made For?", "artist": "Billie Eilish"},
    {"year": 2023, "song": "Just Like That", "artist": "Bonnie Raitt"},
    {"year": 2022, "song": "Leave The Door Open", "artist": "Silk Sonic"},
    {"year": 2021, "song": "Everything I Wanted", "artist": "Billie Eilish"},
    {"year": 2020, "song": "bad guy", "artist": "Billie Eilish"},
    {"year": 2019, "song": "This Is America", "artist": "Childish Gambino"},
    {"year": 2018, "song": "That's What I Like", "artist": "Bruno Mars"},
    {"year": 2017, "song": "Hello", "artist": "Adele"},
    {"year": 2016, "song": "Thinking Out Loud", "artist": "Ed Sheeran"},
    {"year": 2015, "song": "Stay With Me", "artist": "Sam Smith"},
]

print("\n📜 HISTORICAL SOTY WINNERS (2015-2025)")
print("=" * 70)

history_df = pd.DataFrame(HISTORICAL_WINNERS)
display(history_df)

## 7. Aggregate All Data

In [ ]:
# ============================================================================
# AGGREGATE ALL DATA INTO SINGLE STRUCTURE
# ============================================================================

songs_data = {}

for song_name, nominee_info in NOMINEES.items():
    songs_data[song_name] = {
        "metadata": nominee_info,
        "audio": AUDIO_FEATURES.get(song_name, {}),
        "charts": CHART_DATA.get(song_name, {}),
        "grammy": GRAMMY_CONTEXT.get(song_name, {})
    }

print(f"✅ Aggregated data for {len(songs_data)} songs")

# Preview structure
print("\n📋 Data Structure Preview (WILDFLOWER):")
print("-" * 50)
for key, value in songs_data["WILDFLOWER"].items():
    print(f"  {key}: {list(value.keys()) if isinstance(value, dict) else value}")

## 8. Calculate Cultural Impact Score

In [ ]:
# ============================================================================
# CULTURAL IMPACT SCORE CALCULATION
# ============================================================================

def calculate_cultural_impact(charts: dict, grammy: dict) -> float:
    """
    Calculate cultural impact score (0-1) based on:
    - Billboard chart performance (40%)
    - Streaming metrics (30%)
    - Grammy history boost (30%)
    """
    # Billboard score (inverse of peak position, weighted by weeks)
    peak_score = 1 - (charts.get('billboard_peak', 100) / 100)
    weeks_score = min(charts.get('billboard_weeks', 0) / 52, 1.0)  # Normalize to 1 year
    billboard_score = (peak_score * 0.6 + weeks_score * 0.4)
    
    # Streaming score (normalized)
    streams = charts.get('spotify_streams_millions', 0)
    streaming_score = min(streams / 1000, 1.0)  # 1B streams = 1.0
    
    # Grammy history boost
    grammy_score = min(grammy.get('artist_prior_grammy_wins', 0) / 20, 1.0)
    
    # Weighted combination
    cultural_impact = (
        billboard_score * 0.40 +
        streaming_score * 0.30 +
        grammy_score * 0.30
    )
    
    return round(cultural_impact, 3)

# Calculate for all songs
cultural_impact_scores = {}
for song_name, data in songs_data.items():
    score = calculate_cultural_impact(data['charts'], data['grammy'])
    cultural_impact_scores[song_name] = score
    songs_data[song_name]['scores'] = {'cultural_impact': score}

print("\n📊 CULTURAL IMPACT SCORES")
print("=" * 50)
for song, score in sorted(cultural_impact_scores.items(), key=lambda x: x[1], reverse=True):
    bar = "█" * int(score * 30)
    print(f"  {song:<25} {score:.3f} {bar}")

## 9. Save Data

In [ ]:
# ============================================================================
# SAVE COLLECTED DATA
# ============================================================================

# Create data directory
Path('../data').mkdir(exist_ok=True)

# Save nominees
with open('../data/nominees_2026.json', 'w') as f:
    json.dump(NOMINEES, f, indent=2)

# Save audio features
with open('../data/audio_features.json', 'w') as f:
    json.dump(AUDIO_FEATURES, f, indent=2)

# Save chart data
with open('../data/chart_data.json', 'w') as f:
    json.dump(CHART_DATA, f, indent=2)

# Save Grammy context
with open('../data/grammy_context.json', 'w') as f:
    json.dump(GRAMMY_CONTEXT, f, indent=2)

# Save cultural impact scores
with open('../data/cultural_impact_scores.json', 'w') as f:
    json.dump(cultural_impact_scores, f, indent=2)

# Save complete aggregated data
with open('../data/songs_data_complete.json', 'w') as f:
    json.dump(songs_data, f, indent=2)

print("✅ Data saved to ../data/")
print("   • nominees_2026.json")
print("   • audio_features.json")
print("   • chart_data.json")
print("   • grammy_context.json")
print("   • cultural_impact_scores.json")
print("   • songs_data_complete.json")

## Summary

Data collection is complete! We now have:

1. **Audio Features** - Tempo, energy, valence, danceability, acousticness
2. **Chart Performance** - Billboard peak/weeks, streaming metrics
3. **Grammy Context** - Artist history, prior wins, genre bias
4. **Cultural Impact Scores** - Calculated composite scores

### Key Observations:

| Song | Cultural Impact | Notable |
|------|-----------------|----------|
| Die With a Smile | Highest | #1 peak, massive streaming, Gaga+Bruno history |
| WILDFLOWER | High | Longest chart run (71 weeks), Billie's track record |
| APT. | High | #1 peak, Bruno Mars history |
| luther | Medium-High | Kendrick + SZA collab power |

**Next Steps:**
- Notebook 02: Lyrics NLP & Originality Scoring
- Notebook 03: Topic Modeling & Thematic Analysis
- Notebook 04: Audio Feature Deep Dive
- Notebook 05: Final Ensemble Scoring